In [15]:
import tensorflow.keras as keras
import numpy as np
import importlib

import kaggle_functions as kaggle
import model10_wideresnet
importlib.reload(kaggle); 
importlib.reload(model10_wideresnet);
from model10_wideresnet import Model

model_number = 'model10'

### Create datasets

In [5]:
train_dataset, valid_dataset, valid_labels = kaggle.load_train_as_dataset()

batch_size = 32
train_dataset_augmented, epoch_length = kaggle.augment_dataset(train_dataset, batch_size)

In [ ]:
kaggle.show_images(train_dataset_augmented, 9)

### Train new model

In [16]:
Model().build(None).summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 96, 96, 1)]  0           []                               
                                                                                                  
 conv2d_136 (Conv2D)            (None, 48, 48, 16)   160         ['input_8[0][0]']                
                                                                                                  
 batch_normalization_115 (Batch  (None, 48, 48, 16)  64          ['conv2d_136[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 re_lu_115 (ReLU)               (None, 48, 48, 16)   0           ['batch_normalization_115[0

In [17]:
# Fit model
# See {model}.py for specific training instructions, like epochs or valid_patience
model_name = 'WideResNet_6'

model, history = kaggle.train_model(
    Model().build(None), train_dataset.batch(32), valid_dataset, 
    epochs=200, valid_patience=20, epoch_length=epoch_length)

Epoch 1/200
298/298 [==============================] - 182s 587ms/step - loss: 9.8634 - accuracy: 0.2476 - val_loss: 8.3211 - val_accuracy: 0.0841 - lr: 5.0000e-04
Epoch 2/200
298/298 [==============================] - 171s 574ms/step - loss: 5.3821 - accuracy: 0.3139 - val_loss: 4.6055 - val_accuracy: 0.1943 - lr: 5.0000e-04
Epoch 3/200
298/298 [==============================] - 171s 574ms/step - loss: 3.5043 - accuracy: 0.3459 - val_loss: 3.1668 - val_accuracy: 0.3171 - lr: 5.0000e-04
Epoch 4/200
298/298 [==============================] - 171s 574ms/step - loss: 2.8328 - accuracy: 0.3569 - val_loss: 2.9242 - val_accuracy: 0.2876 - lr: 5.0000e-04
Epoch 5/200
298/298 [==============================] - 151s 507ms/step - loss: 2.5471 - accuracy: 0.3919 - val_loss: 2.4973 - val_accuracy: 0.3944 - lr: 5.0000e-04
Epoch 6/200
298/298 [==============================] - 139s 466ms/step - loss: 2.4299 - accuracy: 0.4078 - val_loss: 2.9388 - val_accuracy: 0.3267 - lr: 5.0000e-04
Epoch 7/200
298/

In [ ]:
# Save model
model.save(f'models/{model_number}/{model_name}')

# Plot model statistics during training
kaggle.plot_model_history(history, [['accuracy', 'val_accuracy'], ['loss', 'val_loss']])

### Fine-tune

In [9]:
# Fine-tune model
fine_model, history = kaggle.fine_tune_model(
    model,
    train_dataset.batch(32), valid_dataset, 
    epochs=2, epoch_length=epoch_length, learning_rate=1E-5)

Epoch 1/2
298/298 [==============================] - 143s 458ms/step - loss: 1.7360 - accuracy: 0.9830 - val_loss: 2.3162 - val_accuracy: 0.8764
Epoch 2/2
298/298 [==============================] - 134s 451ms/step - loss: 1.7383 - accuracy: 0.9842 - val_loss: 2.3240 - val_accuracy: 0.8751


In [16]:
# Save fine-tuned model
fine_model_name = 'WideResNet_5_84'
fine_model.save(f'models/{model_number}/{fine_model_name}')

INFO:tensorflow:Assets written to: models/model10/WideResNet_5_84\assets


C:\Users\Guillaume\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\Guillaume\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


### Evaluate

In [ ]:
model_name = ''
model = keras.models.load_model(f'models/{model_number}/{model_name}')
test_pred = np.argmax(model.predict(valid_dataset.batch(128)), axis=1)

kaggle.print_accuracy(valid_labels, test_pred)
kaggle.plot_confusion_matrix(valid_labels, test_pred)

### Generate test predictions

In [ ]:
model_name = 'WideResNet_3'
kaggle.generate_test_pred_filepath(f'models/{model_number}/{model_name}')

In [11]:
model_name = 'Stacking_0'
kaggle.generate_test_pred(model, f'models/{model_number}/{model_name}_test_pred.csv')

### Hypertune models

In [ ]:
tuner = kaggle.hypertune_model(
    Model(), train_dataset.batch(32).cache(), valid_dataset, 
    model_number, 'hypertuner2021-12-03', trials=4, 
    epochs=11, valid_patience=3, epoch_length=epoch_length)

In [4]:
tuner = kaggle.load_hypertuner(Model(), model_number, 'hypertuner2021-12-03')

INFO:tensorflow:Reloading Oracle from existing project models/model10\hypertuner2021-12-03\oracle.json
INFO:tensorflow:Reloading Tuner from models/model10\hypertuner2021-12-03\tuner0.json
Results summary
Results in models/model10\hypertuner2021-12-03
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_dropout: 0.5
k: 9
Score: 0.8048780560493469
Trial summary
Hyperparameters:
conv_dropout: 0.3
k: 9
Score: 0.8015138506889343
Trial summary
Hyperparameters:
conv_dropout: 0.5
k: 6
Score: 0.7211942672729492


In [10]:
model = tuner.get_best_models(1)[0]